# WeatherPy
----

### Analysis
* As expected, the weather becomes significantly warmer as one approaches the equator (0 Deg. Latitude). More interestingly, however, is the fact that the southern hemisphere tends to be warmer this time of year than the northern hemisphere. This may be due to the tilt of the earth.
* There is no strong relationship between latitude and cloudiness. However, it is interesting to see that a strong band of cities sits at 0, 80, and 100% cloudiness.
* There is no strong relationship between latitude and wind speed. However, in northern hemispheres there is a flurry of cities with over 20 mph of wind.

---

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [37]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import json

# Import API key
from api_keys import myKey

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

## Generate Cities List

In [2]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []
countries = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(low=-90.000, high=90.000, size=1500)  
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)
lat_lngs = zip(lats, lngs)
#uniform is to have the numbers evenly distributed
#size is how many random numbers will be generated.
#we are saying, generate 1500 random evenly distributed numbers between -90 and 90.

# Identify nearest city for each lat, lng combination
for each in lat_lngs:
    city = citipy.nearest_city(each[0], each[1]).city_name 
    country = citipy.nearest_city(each[0], each[1]).country_code 
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)
        countries.append(country)

# Print the city count to confirm sufficient count
len(cities)

#612 cites are returned.   There must be at least 500 on the next part of finding the weather.  If not, then must come back to this step and increase size.

589

In [3]:
len(countries)

589

In [4]:
#??  how do I see the ln_lngs list?
print(lat_lngs)

In [5]:
print(lngs)

[  70.31702327  -47.13330494   83.70620553 ...  127.85508736 -135.46144258
  -21.92338739]


In [6]:
#

### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [7]:
#first, perform a check on a single city.

In [8]:
url = "http://api.openweathermap.org/data/2.5/weather?"
city = "alofi"
units = "imperial"

# Build query URL
query_url = url + "appid=" + myKey + "&q=" + city + "&units=imperial"
#note:   adding imperial units will return the temperature in farenheight and the wind speed in miles per hour
print(query_url)

http://api.openweathermap.org/data/2.5/weather?appid=8bea25d5e18b7c2fb2ee3ac12cec5dad&q=alofi&units=imperial


In [9]:
# Get weather data
weather_response = requests.get(query_url)
weather_json = weather_response.json()


In [10]:
print(json.dumps(weather_json, indent=4, sort_keys=True))

{
    "base": "stations",
    "clouds": {
        "all": 44
    },
    "cod": 200,
    "coord": {
        "lat": -19.06,
        "lon": -169.92
    },
    "dt": 1554048000,
    "id": 4036284,
    "main": {
        "humidity": 94,
        "pressure": 1009,
        "temp": 78.8,
        "temp_max": 78.8,
        "temp_min": 78.8
    },
    "name": "Alofi",
    "rain": {
        "3h": 0.142
    },
    "sys": {
        "country": "NU",
        "id": 7306,
        "message": 0.0036,
        "sunrise": 1554053160,
        "sunset": 1554096080,
        "type": 1
    },
    "weather": [
        {
            "description": "light rain",
            "icon": "10n",
            "id": 500,
            "main": "Rain"
        }
    ],
    "wind": {
        "deg": 290,
        "speed": 5.82
    }
}


In [11]:
#Success!   Able to retrieve a single city.
#now retrieve each individual piece of data needed for the loop

In [12]:
print(weather_json["name"])

Alofi


In [13]:
print(weather_json["coord"]["lat"])

-19.06


In [14]:
print(weather_json["main"]["temp"])

78.8


In [15]:
print(weather_json["main"]["humidity"])

94


In [16]:
print(weather_json["clouds"]["all"])

44


In [17]:
print(weather_json["wind"]["speed"])

5.82


### need a dataframe containing city and country to begin
NOTE:   the Weather API will take only a city; however, if multiples are found, API responds with a list of results that match a searching word. 
<BR>
Therefore, I will begin with a list of cities and countries which can also be passed to the 
    <BR>q city name and country code divided by comma, use ISO 3166 country codes
        
<BR> https://openweathermap.org/current

In [18]:
#Begin with the city and country list which was created in the begining.

cityCountry = pd.DataFrame(list(zip(cities, countries)),
              columns=['city','country'])

cityCountry.head()

,city,country
0,atasu,kz
1,torbay,ca
2,kattivakkam,in
3,atuona,pf
4,penzance,gb


In [19]:
#confirm there are at least 500 countries
len(cityCountry)

589

In [20]:
#  ??? not sure if I will take this approach

#the weather API allows for no more than 60 calls per minutes.
#divide the list into groups of 60
#https://openweathermap.org/price

# Add column for batch
# Note that we used "" to specify initial entry.
cityCountry["batch"] = ""
cityCountry.head()

,city,country,batch
0,atasu,kz,
1,torbay,ca,
2,kattivakkam,in,
3,atuona,pf,
4,penzance,gb,


In [21]:
# create a data frome with the values needed for the API calls

cityCountry2 = cityCountry
cityCountry2["location"] = (cityCountry2["city"] + ", " + cityCountry2["country"])
cityCountry2["newCity"] = ""
cityCountry2["newCountry"] = ""
cityCountry2["lat"] = ""
cityCountry2["long"] = ""
cityCountry2["tempF"] = ""
cityCountry2["humidity"] = ""
cityCountry2["cloud"] = ""
cityCountry2["windMPH"] = ""
cityCountry2["maxTemp"] = ""
cityCountry2.head()

In [22]:
cityCountry2.head()

,city,country,batch,location,newCity,newCountry,lat,long,tempF,humidity,cloud,windMPH,maxTemp
0,atasu,kz,,"atasu, kz",,,,,,,,,
1,torbay,ca,,"torbay, ca",,,,,,,,,
2,kattivakkam,in,,"kattivakkam, in",,,,,,,,,
3,atuona,pf,,"atuona, pf",,,,,,,,,
4,penzance,gb,,"penzance, gb",,,,,,,,,


### Populate the data only for a few records
Practice before attemping to get data for 500+ cities

In [24]:
#create a new df with only 10 records
cityCountrySample = cityCountry2.head(10)
cityCountrySample.head(20)

,city,country,batch,location,newCity,newCountry,lat,long,tempF,humidity,cloud,windMPH,maxTemp
0,atasu,kz,,"atasu, kz",,,,,,,,,
1,torbay,ca,,"torbay, ca",,,,,,,,,
2,kattivakkam,in,,"kattivakkam, in",,,,,,,,,
3,atuona,pf,,"atuona, pf",,,,,,,,,
4,penzance,gb,,"penzance, gb",,,,,,,,,
5,rikitea,pf,,"rikitea, pf",,,,,,,,,
6,kodiak,us,,"kodiak, us",,,,,,,,,
7,ushuaia,ar,,"ushuaia, ar",,,,,,,,,
8,cape town,za,,"cape town, za",,,,,,,,,
9,upernavik,gl,,"upernavik, gl",,,,,,,,,


In [42]:
from api_keys import api_key

In [43]:
parameters = {"appid": api_key,
             "units": "imperial",
             "city": "alofi"}
base_url = "http://api.openweathermap.org/data/2.5/weather?"

In [44]:
response = requests.get(base_url, params=parameters)

In [45]:
print(response.url)

http://api.openweathermap.org/data/2.5/weather?appid=8bea25d5e18b7c2fb2ee3ac12cec5dad&units=imperial&city=alofi


In [ ]:
# create a params dict that will be updated with new city each iteration
parameters = {"key": api_key,
             "units": "&units=imperial"}

# Loop through the cityCountrySample and run a search on each
for index, row in cityCountrySample.iterrows():
    base_url = "http://api.openweathermap.org/data/2.5/weather?"

    city = row['City']
    state = row['State']

    # update address key value
    params['address'] = f"{city},{state}"

    # make request
    cities_lat_lng = requests.get(base_url, params=params)
    
    # print the cities_lat_lng url, avoid doing for public github repos in order to avoid exposing key
    # print(cities_lat_lng.url)
    
    # convert to json
    cities_lat_lng = cities_lat_lng.json()

    cities_pd.loc[index, "Lat"] = cities_lat_lng["results"][0]["geometry"]["location"]["lat"]
    cities_pd.loc[index, "Lng"] = cities_lat_lng["results"][0]["geometry"]["location"]["lng"]

# Visualize to confirm lat lng appear
cities_pd.head()


### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

### Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

#### Latitude vs. Temperature Plot

#### Latitude vs. Humidity Plot

#### Latitude vs. Cloudiness Plot

#### Latitude vs. Wind Speed Plot